In [1]:
import huggingface_hub
import datasets
import os

HUGGINGFACE_TOKEN = os.environ.get("HUGGINGFACE_TOKEN")

huggingface_hub.login(HUGGINGFACE_TOKEN)

In [2]:
dataset = datasets.load_dataset("kahua-ml/flattened_nameplate_dataset", split="train")

# Model Loading

In [3]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
def inference(img, text):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": img,
                },
                {"type": "text", "text": text},
            ],
        }
    ]
    # Preparation for inference
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return(output_text[0])


In [5]:
def text_formatter(field):
    text = f"You are a helpful assistant that extracts text from images and returns JSON for programmatic use. Find the value of {field} and return a JSON string in the format of {{'{field}': '<value>'}}."
    return text

In [15]:
def answer_formatter(field, value):
    return f"{{'{field}': '{value}'}}"

Loading Peft

In [10]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=64,  # Larger rank is often better for large models
    lora_alpha=128,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Specific to Qwen's attention/FFN
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model.enable_input_require_grads()  # Required for LoRA
peft_model = get_peft_model(model, peft_config)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(


Dataset Split

In [11]:
# Perform an 80-20 random split
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Assign to separate variables
train_ds = split_dataset["train"]
test_ds = split_dataset["test"]

train_ds = train_ds.shuffle()
test_ds = test_ds.shuffle()

In [16]:
from typing import Any
from unittest.util import _MAX_LENGTH
import torch
from qwen_vl_utils import process_vision_info


class MyDataCollator:
    def __init__(self, processor) -> None:
        self.processor = processor
       
    def __call__(self, examples) -> Any:
        texts = []
        images = []
        assistant_responses = []

        for example in examples:
            image = example["image"]
            field = example["field"]
            value = example["value"]
            answer = answer_formatter(field, value)
           
            messages = [
                {
                    "role": "user",
                    "content": [
                        # If you want EXACT order as inference, put image first
                        {"type": "image", "image": image},
                        {"type": "text", "text": text_formatter(field)},
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": answer}
                    ]
                }
            ]
           
            # Make sure to unify arguments with inference
            text = self.processor.apply_chat_template(
                messages, 
                tokenize=False,                 # match inference
                add_generation_prompt=True       # match inference
            )
            texts.append(text)
            images += process_vision_info(messages)[0]
            assistant_responses.append(answer)
       
        batch = self.processor(
            text=texts,
            images=images,
            return_tensors='pt',
            padding=True,
        )

        labels = batch["input_ids"].clone()
       
        for i, (input_ids, assistant_response) in enumerate(zip(batch["input_ids"], assistant_responses)):
            assistant_tokens = self.processor.tokenizer(
                assistant_response, return_tensors="pt"
            )["input_ids"][0]
            
            start_idx = self.find_subsequence(input_ids, assistant_tokens)
            if start_idx is not None:
                labels[i, :start_idx] = -100
                labels[i, start_idx + len(assistant_tokens):] = -100

        batch["labels"] = labels
        return batch

    def find_subsequence(self, sequence, subsequence):
        seq_len = len(sequence)
        sub_len = len(subsequence)
       
        for i in range(seq_len - sub_len + 1):
            if torch.equal(sequence[i : i + sub_len], subsequence):
                return i
        return None
   
data_collator = MyDataCollator(processor)  

In [18]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
    num_train_epochs=1,  # You can go longer, but monitor for overfitting
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,  # Helps simulate larger batch size
    learning_rate=1e-4,  # Smaller LR for larger models
    warmup_ratio=0.03,
    logging_steps=80,
    eval_strategy="steps",
    eval_steps=80,
    save_strategy="steps",
    save_steps=80,
    save_total_limit=2,
    gradient_checkpointing=True,
    bf16=True,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    remove_unused_columns=False,
    output_dir="./qwen-peft-vl",
    report_to="none",  # Avoid external logging unless needed
)


trainer = Trainer(
    model=peft_model,
    args=training_args,
    data_collator=data_collator,
    eval_dataset=test_ds,
    train_dataset=train_ds,
)

In [19]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss
80,0.206000,0.131596
160,0.124100,0.110980


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=176, training_loss=0.176583086902445, metrics={'train_runtime': 1184.1284, 'train_samples_per_second': 0.297, 'train_steps_per_second': 0.149, 'total_flos': 1.852223462551757e+16, 'train_loss': 0.176583086902445, 'epoch': 1.0})